## Optical flow

First take in a given set of points and a frame. Then it attempts to find those points in the next frame. The user should give the points to track.

We pass in the previous frame, previous points and the current frame to the **Lucas-Kanade** function.

The function computes optical flow for a **sparse** feature set. (only points that are told to be tracked)

Track all points in the video: **Gunner Farneback's** algorithm: calculate **dense** optical flow. (all points in an image)

### Lucas Kanade function

In [1]:
import numpy as np
import cv2

First detect corners

In [2]:
corner_track_params = dict(maxCorners=10, qualityLevel=0.3,
                          minDistance=7, blockSize=7)


`maxLevel` is for pyramid in image processing.

We will two criterion for the L-K function.

In [3]:
lk_params = dict(winSize=(200, 200), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [6]:
cap = cv2.VideoCapture(0)

ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# points to track
prev_pts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)
mask = np.zeros_like(prev_frame)

while True:
    ret, frame = cap.read()
    cur_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Call Lukas Kanade with pyramid
    next_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, cur_gray, prev_pts, None, **lk_params)
    
    # status == 1 if the next point is found in the new frame
    good_new = next_pts[status == 1]
    good_prev = prev_pts[status == 1]
    
    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        x_new, y_new = new.ravel() # Flatten the array
        x_prev, y_prev = prev.ravel()
        mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev)
                       , (0, 255, 0), 3)
        
        frame = cv2.circle(frame, (x_new, y_new), 8, (0, 0, 255), -1)
    
    img = cv2.add(frame, mask)
    cv2.imshow('tracking', img)
    
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
    
    prev_gray = cur_gray.copy()
    prev_pts = good_new.reshape(-1, 1, 2)
    
cv2.destroyAllWindows()
cap.release()